In [15]:
import pymongo
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [16]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb=myclient['smartshark_2_1']
mycol = mydb["project"]

projects=[]
list2=[]
list3=[]
list4=[]
detailedissues=[]
joinedissues = mydb.project.aggregate([{"$lookup":{
            "from": "issue_system",       # other table name
            "localField": "_id",        # key field in collection 2
            "foreignField": "project_id",      # key field in collection 1
            "as": "linked_collections"   # alias for resulting table
        }},
        {   "$unwind":"$linked_collections"},
                                
         {"$lookup":{
            "from": "issue",       # other table name
            "localField": "linked_collections._id",        # key field in collection 2
            "foreignField": "issue_system_id",      # key field in collection 1
            "as": "linked_collections2"   # alias for resulting table
        }},
         {   "$unwind":"$linked_collections2" }
                 , {"$lookup":{
            "from": "issue_comment",       # other table name
            "localField": "linked_collections2._id",        # key field in collection 2
            "foreignField": "issue_id",      # key field in collection 1
            "as": "linked_collections3"   # alias for resulting table
        }},
         {   "$unwind":"$linked_collections3" },                             
        {"$project": {"_id":1,"name":1,"linked_collections3.comment":1}}
        ])


listofissues=[]

for docs in joinedissues:
      list3.append((docs["_id"],docs["name"],docs["linked_collections3"]))
      

for project in mycol.find():
 projects.append(project["_id"])


In [17]:
projectissue=[]
for projectid in projects:
    detailedissue=[]
  

    hit1=0
    for item,name,comment in list3:
        
      
        if projectid==item:
            hit1=hit1+1
            detailedissue.append(comment)
    projectissue.append(detailedissue)

    listofissues.append(hit1)


In [ ]:
from nltk.corpus import names
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import string
from sklearn.feature_extraction import text

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
import pandas as pd
import gensim.corpora as corpora
from nltk import word_tokenize, pos_tag


lemmatizer = WordNetLemmatizer()

ignore = ['{','}','.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%",'\n','\n2','ok','yes','no','thanks','good','sure','did','Thanks','Sure','welcome']  
stop_words = text.ENGLISH_STOP_WORDS.union(ignore)
vectorizer = TfidfVectorizer(stop_words = stop_words)
cv = CountVectorizer(stop_words=stop_words)

cleaned=[]
def letters_only(astr):
 return astr.isalpha()

exclude = set(string.punctuation)


listofsentenes=[]
import re
import string




def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

round1 = lambda x: clean_text_round1(x)
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2]=='VB'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)
def isoneword(text):
    if len(text)>2:
        return True
    else:
        return False




for test in projectissue:
 for post in test:
     
    
     values=post['comment']
     
     
     sentence=nouns_adj(values)
     
     cleaned.append(' '.join([lemmatizer.lemmatize(word.lower()) for word in sentence.split() if  letters_only(word) and isoneword(word) and word not in stop_words and word not in exclude and word not in ignore]))

   

transformed = cv.fit_transform(cleaned)


In [8]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
import pickle
from sklearn.feature_extraction.text import CountVectorizer
list1=pickle.load(open('model.pk4','rb'))
cv = CountVectorizer()
transformed = cv.fit_transform(list1)
lda = LatentDirichletAllocation(n_components=10, random_state=42).fit(transformed)
for topic_idx, topic in enumerate(lda.components_):
 label = '{}: '.format(topic_idx)
 print(label, " ".join([cv.get_feature_names_out()[i]
 for i in topic.argsort()[:-10:-1]]))

0:  think code case need make use change data doe
1:  code file use version work class dependency using jar
2:  issue fixed fix http patch jira look change commit
3:  http pull request commit user github date author close
4:  patch test http warning trunk doe committed unit exec
5:  string final new public void return import null private
6:  val pogo new support game def int number help
7:  info error code test run job failure exception failed
8:  broker partition topic log message offset consumer request kafka
9:  license file see distributed apache software http asf diff


In [11]:
from textblob import TextBlob

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity
subresults=[]
polresults=[]
for test in projectissue:
 for post in test:
     values=post['comment']
     polarity=pol(values)
     subjectivity=sub(values)
     subresults.append(subjectivity)
     polresults.append(polarity)

one=np.asarray(subresults)
two=np.asarray(polresults)
print(np.mean(one,axis=None))
print(np.mean(two,axis=None))

0.31984428297556333
0.07111781112644425


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(cleaned)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print


Top terms per cluster:
Cluster 0:
 committed
 patch
 color
 trunk
 master
 applied
 green
 warning
 test
 revision
Cluster 1:
 http
 patch
 issue
 fixed
 test
 branch
 code
 file
 fix
 change
Cluster 2:
 pull
 http
 request
 github
 user
 closed
 asfgit
 commented
 issue
 resolved


In [9]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=20, random_state=42).fit(transformed)
for topic_idx, topic in enumerate(nmf.components_):
 label = '{}: '.format(topic_idx)
 print(label, " ".join([cv.get_feature_names_out()[i]
 for i in topic.argsort()[:-9:-1]]))

D:\dev\datamining\venv\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


0:  license distributed file software apache see asf foundation
1:  final string topic link new public null specified
2:  final new string public byte void private key
3:  pogo game support help customer number quick contact
4:  info talledlocalcontainer case return final key public relclasses
5:  new return public schema version static private throttletimems
6:  number quickbooks support phone sage customer pro help
7:  commit http author date pull change request doe
8:  val new def topic string import message class
9:  java socket unix cyclone stream implicit record code
10:  info log offset broker completed end load truncating
11:  link record string final key code ktable value
12:  public string new void return throw import exception
13:  import expectedkey final new string time private expectedkeys
14:  info exists destination writing test skipped run failure
15:  starttime assertequals windowsize tolist incr final void key
16:  ivy retrieve dependency resolving default file ant c

In [12]:
from tqdm import tqdm
import gensim

import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS

import pprint
final=[doc.split() for doc in cleaned]
dictionary = gensim.corpora.Dictionary(final)
dictionary.filter_extremes(no_below=15, no_above=0.4,keep_n=100000)
corpus = [dictionary.doc2bow(doc) for doc in final]


lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=42,
                                           update_every=1,
                                           
                                           passes=40,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.print_topics()



[(0,
  '0.332*"http" + 0.142*"user" + 0.141*"pull" + 0.134*"request" + 0.063*"github" + 0.039*"closed" + 0.017*"asfgit" + 0.008*"window" + 0.007*"arg" + 0.007*"cli"'),
 (1,
  '0.155*"issue" + 0.089*"release" + 0.078*"resolved" + 0.075*"fix" + 0.071*"fixed" + 0.068*"version" + 0.039*"master" + 0.035*"bug" + 0.029*"updated" + 0.020*"thank"'),
 (2,
  '0.025*"code" + 0.017*"case" + 0.016*"need" + 0.015*"think" + 0.014*"work" + 0.013*"look" + 0.012*"change" + 0.012*"make" + 0.011*"problem" + 0.011*"time"'),
 (3,
  '0.100*"info" + 0.082*"error" + 0.055*"falcon" + 0.033*"entity" + 0.032*"cluster" + 0.028*"server" + 0.020*"success" + 0.017*"main" + 0.017*"log" + 0.016*"noformat"'),
 (4,
  '0.101*"patch" + 0.076*"test" + 0.041*"number" + 0.033*"file" + 0.028*"kylin" + 0.024*"doe" + 0.020*"git" + 0.019*"build" + 0.015*"new" + 0.014*"run"'),
 (5,
  '0.052*"game" + 0.020*"add" + 0.019*"comment" + 0.018*"method" + 0.017*"use" + 0.016*"new" + 0.016*"property" + 0.015*"set" + 0.015*"class" + 0.013*"i

In [14]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=final, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.520833895688165


In [16]:
import pickle


# pickle.dump(cleaned, open('model.pk4','wb'))
# pickle.dump(dictionary, open('model.pk5','wb'))
# pickle.dump(corpus, open('model.pk5','wb'))
pickle.dump(lda_model, open('model.pkn100000,.4,10','wb'))
# Loading model to compare the results
model = pickle.load(open('model.pkn100000,.4,10','rb'))
model.print_topics()


[(0,
  '0.332*"http" + 0.142*"user" + 0.141*"pull" + 0.134*"request" + 0.063*"github" + 0.039*"closed" + 0.017*"asfgit" + 0.008*"window" + 0.007*"arg" + 0.007*"cli"'),
 (1,
  '0.155*"issue" + 0.089*"release" + 0.078*"resolved" + 0.075*"fix" + 0.071*"fixed" + 0.068*"version" + 0.039*"master" + 0.035*"bug" + 0.029*"updated" + 0.020*"thank"'),
 (2,
  '0.025*"code" + 0.017*"case" + 0.016*"need" + 0.015*"think" + 0.014*"work" + 0.013*"look" + 0.012*"change" + 0.012*"make" + 0.011*"problem" + 0.011*"time"'),
 (3,
  '0.100*"info" + 0.082*"error" + 0.055*"falcon" + 0.033*"entity" + 0.032*"cluster" + 0.028*"server" + 0.020*"success" + 0.017*"main" + 0.017*"log" + 0.016*"noformat"'),
 (4,
  '0.101*"patch" + 0.076*"test" + 0.041*"number" + 0.033*"file" + 0.028*"kylin" + 0.024*"doe" + 0.020*"git" + 0.019*"build" + 0.015*"new" + 0.014*"run"'),
 (5,
  '0.052*"game" + 0.020*"add" + 0.019*"comment" + 0.018*"method" + 0.017*"use" + 0.016*"new" + 0.016*"property" + 0.015*"set" + 0.015*"class" + 0.013*"i

In [6]:
new_list = [s.replace("code", "") for s in cleaned]
final=[doc.split() for doc in new_list]
dictionary = gensim.corpora.Dictionary(final)

dictionary.filter_extremes(no_below=15, no_above=0.3,keep_n=100000)
corpus = [dictionary.doc2bow(doc) for doc in final]


lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=20, 
                                           random_state=42,
                                           update_every=1,
                                           
                                           passes=40,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.print_topics()

[(0,
  '0.241*"pull" + 0.229*"request" + 0.120*"http" + 0.084*"user" + 0.067*"closed" + 0.064*"close" + 0.064*"apply" + 0.062*"running" + 0.007*"assign" + 0.006*"incorporated"'),
 (1,
  '0.232*"fix" + 0.204*"fixed" + 0.101*"bug" + 0.059*"thank" + 0.044*"delete" + 0.032*"checked" + 0.031*"could" + 0.030*"post" + 0.029*"execution" + 0.022*"reviewing"'),
 (2,
  '0.188*"test" + 0.116*"number" + 0.048*"run" + 0.040*"result" + 0.038*"cluster" + 0.030*"unit" + 0.029*"success" + 0.027*"failure" + 0.025*"time" + 0.017*"testing"'),
 (3,
  '0.255*"help" + 0.234*"release" + 0.088*"quick" + 0.037*"filter" + 0.034*"name" + 0.020*"document" + 0.017*"resolve" + 0.016*"conflict" + 0.014*"response" + 0.014*"sound"'),
 (4,
  '0.066*"type" + 0.058*"method" + 0.044*"class" + 0.042*"used" + 0.040*"added" + 0.027*"function" + 0.025*"api" + 0.021*"field" + 0.020*"object" + 0.018*"expression"'),
 (5,
  '0.090*"diff" + 0.064*"index" + 0.045*"copy" + 0.043*"additional" + 0.041*"language" + 0.037*"limitation" + 0

In [10]:
pickle.dump(lda_model, open('model.pk6','wb'))

In [11]:
cleaned = pickle.load(open('model.pk4','rb'))

final=[doc.split() for doc in cleaned]
dictionary = gensim.corpora.Dictionary(final)

dictionary.filter_extremes(no_below=15, no_above=0.4,keep_n=800000)
corpus = [dictionary.doc2bow(doc) for doc in final]


lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=11, 
                                           random_state=42,
                                           update_every=1,
                                           
                                           passes=40,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.print_topics()

[(0,
  '0.210*"user" + 0.210*"pull" + 0.199*"request" + 0.061*"cluster" + 0.058*"closed" + 0.026*"asfgit" + 0.023*"commented" + 0.018*"alternatively" + 0.012*"window" + 0.011*"bvellanki"'),
 (1,
  '0.427*"http" + 0.103*"resolved" + 0.094*"fixed" + 0.081*"github" + 0.081*"git" + 0.052*"master" + 0.023*"branch" + 0.021*"contribution" + 0.015*"closing" + 0.009*"duplicate"'),
 (2,
  '0.027*"code" + 0.016*"case" + 0.015*"use" + 0.013*"think" + 0.010*"change" + 0.010*"make" + 0.009*"table" + 0.009*"using" + 0.009*"time" + 0.008*"add"'),
 (3,
  '0.073*"cube" + 0.071*"query" + 0.021*"noformat" + 0.020*"tuple" + 0.019*"count" + 0.019*"load" + 0.018*"record" + 0.016*"bag" + 0.015*"data" + 0.015*"byte"'),
 (4,
  '0.074*"test" + 0.041*"number" + 0.029*"file" + 0.027*"kylin" + 0.020*"doe" + 0.018*"build" + 0.017*"new" + 0.017*"run" + 0.014*"pig" + 0.014*"release"'),
 (5,
  '0.130*"info" + 0.106*"error" + 0.033*"phone" + 0.032*"exception" + 0.029*"calcite" + 0.023*"time" + 0.022*"main" + 0.017*"meta

In [18]:
cleaned = pickle.load(open('model.pk4','rb'))



new_list = [s.replace("code", "") for s in cleaned]

final=[doc.split() for doc in new_list]
dictionary = gensim.corpora.Dictionary(final)

dictionary.filter_extremes(no_below=15, no_above=0.4,keep_n=200000)
corpus = [dictionary.doc2bow(doc) for doc in final]


lda_model2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=42,
                                           update_every=1,
                                           
                                           passes=40,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model2.print_topics()

[(0,
  '0.150*"http" + 0.093*"commit" + 0.066*"pull" + 0.062*"request" + 0.055*"date" + 0.050*"author" + 0.047*"review" + 0.044*"user" + 0.037*"branch" + 0.029*"github"'),
 (1,
  '0.094*"table" + 0.063*"entity" + 0.047*"join" + 0.023*"delete" + 0.022*"thread" + 0.018*"state" + 0.016*"partition" + 0.016*"process" + 0.015*"execution" + 0.013*"event"'),
 (2,
  '0.112*"test" + 0.063*"number" + 0.038*"falcon" + 0.035*"git" + 0.022*"cluster" + 0.022*"message" + 0.020*"contact" + 0.019*"server" + 0.018*"unit" + 0.017*"success"'),
 (3,
  '0.047*"value" + 0.043*"cube" + 0.042*"query" + 0.041*"property" + 0.025*"select" + 0.023*"input" + 0.020*"type" + 0.020*"set" + 0.019*"hdfs" + 0.019*"default"'),
 (4,
  '0.028*"support" + 0.026*"game" + 0.018*"help" + 0.015*"case" + 0.014*"use" + 0.013*"think" + 0.012*"look" + 0.010*"make" + 0.010*"change" + 0.010*"need"'),
 (5,
  '0.052*"null" + 0.048*"string" + 0.046*"return" + 0.040*"new" + 0.034*"public" + 0.027*"feed" + 0.024*"int" + 0.023*"throw" + 0.02

In [10]:
from tqdm import tqdm
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
cleaned = pickle.load(open('model.pk4','rb'))

final=[doc.split() for doc in cleaned]
dictionary = gensim.corpora.Dictionary(final)

new_list = [s.replace("code", "") for s in cleaned]
newlist1=[s.replace("http","") for s in newlist]


final=[doc.split() for doc in new_list1]
dictionary = gensim.corpora.Dictionary(final)

dictionary.filter_extremes(no_below=15, no_above=0.4,keep_n=200000)
corpus = [dictionary.doc2bow(doc) for doc in final]


lda_model2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           
                                           passes=40,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model2.print_topics()

[(0,
  '0.115*"date" + 0.049*"server" + 0.038*"java" + 0.027*"client" + 0.019*"record" + 0.018*"will" + 0.017*"debug" + 0.015*"target" + 0.015*"recipe" + 0.015*"link"'),
 (1,
  '0.032*"support" + 0.029*"game" + 0.020*"help" + 0.019*"need" + 0.018*"change" + 0.016*"use" + 0.015*"think" + 0.014*"work" + 0.013*"customer" + 0.013*"make"'),
 (2,
  '0.072*"null" + 0.057*"string" + 0.047*"return" + 0.042*"new" + 0.040*"public" + 0.029*"int" + 0.028*"throw" + 0.025*"final" + 0.023*"object" + 0.020*"value"'),
 (3,
  '0.062*"license" + 0.058*"git" + 0.055*"github" + 0.047*"apache" + 0.045*"user" + 0.035*"closed" + 0.034*"file" + 0.032*"http" + 0.030*"date" + 0.025*"distributed"'),
 (4,
  '0.180*"patch" + 0.125*"test" + 0.043*"review" + 0.032*"committed" + 0.029*"new" + 0.028*"doe" + 0.023*"trunk" + 0.021*"number" + 0.020*"unit" + 0.020*"warning"'),
 (5,
  '0.089*"http" + 0.083*"issue" + 0.048*"release" + 0.042*"resolved" + 0.041*"fix" + 0.038*"fixed" + 0.036*"version" + 0.030*"jira" + 0.021*"bui

In [7]:
from tqdm import tqdm
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
cleaned = pickle.load(open('model.pk4','rb'))

final=[doc.split() for doc in cleaned]
dictionary = gensim.corpora.Dictionary(final)

new_list = [s.replace("code", "") for s in cleaned]
new_list1=[s.replace("http","") for s in new_list]

new_list2=[s.replace("asf","") for s in new_list1]
final=[doc.split() for doc in new_list1]
dictionary = gensim.corpora.Dictionary(final)

dictionary.filter_extremes( no_above=0.4,keep_n=1200000)
corpus = [dictionary.doc2bow(doc) for doc in final]


lda_model2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=0,
                                           
                                           passes=40,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model2.print_topics()

[(0,
  '0.093*"info" + 0.065*"error" + 0.014*"failed" + 0.013*"test" + 0.013*"failure" + 0.012*"branch" + 0.012*"success" + 0.012*"reviewboard" + 0.010*"debug" + 0.010*"run"'),
 (1,
  '0.033*"patch" + 0.023*"test" + 0.010*"doe" + 0.009*"new" + 0.008*"change" + 0.007*"file" + 0.007*"warning" + 0.006*"trunk" + 0.006*"release" + 0.006*"work"'),
 (2,
  '0.105*"license" + 0.042*"file" + 0.037*"distributed" + 0.031*"apache" + 0.027*"software" + 0.027*"see" + 0.025*"asf" + 0.019*"diff" + 0.017*"language" + 0.016*"licensed"'),
 (3,
  '0.017*"max" + 0.013*"sum" + 0.013*"min" + 0.013*"connector" + 0.012*"commit" + 0.012*"speed" + 0.011*"mean" + 0.011*"rate" + 0.011*"duplicate" + 0.009*"moon"'),
 (4,
  '0.024*"value" + 0.014*"key" + 0.013*"class" + 0.013*"method" + 0.012*"link" + 0.011*"default" + 0.011*"use" + 0.010*"used" + 0.010*"ivy" + 0.009*"property"'),
 (5,
  '0.032*"issue" + 0.021*"user" + 0.015*"version" + 0.012*"commented" + 0.011*"github" + 0.009*"build" + 0.009*"need" + 0.009*"file" +

In [8]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=final, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.599150531105369


In [13]:
new_list2=[s.replace("asf","") for s in new_list1]
final=[doc.split() for doc in new_list1]
dictionary = gensim.corpora.Dictionary(final)

dictionary.filter_extremes( no_above=0.4,keep_n=1250000)
corpus = [dictionary.doc2bow(doc) for doc in final]

lda_model3 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=0,
                                           
                                           passes=40,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model3.print_topics()

[(0,
  '0.093*"info" + 0.065*"error" + 0.014*"failed" + 0.013*"test" + 0.013*"failure" + 0.012*"branch" + 0.012*"success" + 0.012*"reviewboard" + 0.010*"debug" + 0.010*"run"'),
 (1,
  '0.033*"patch" + 0.023*"test" + 0.010*"doe" + 0.009*"new" + 0.008*"change" + 0.007*"file" + 0.007*"warning" + 0.006*"trunk" + 0.006*"release" + 0.006*"work"'),
 (2,
  '0.105*"license" + 0.042*"file" + 0.037*"distributed" + 0.031*"apache" + 0.027*"software" + 0.027*"see" + 0.025*"asf" + 0.019*"diff" + 0.017*"language" + 0.016*"licensed"'),
 (3,
  '0.017*"max" + 0.013*"sum" + 0.013*"min" + 0.013*"connector" + 0.012*"commit" + 0.012*"speed" + 0.011*"mean" + 0.011*"rate" + 0.011*"duplicate" + 0.009*"moon"'),
 (4,
  '0.024*"value" + 0.014*"key" + 0.013*"class" + 0.013*"method" + 0.012*"link" + 0.011*"default" + 0.011*"use" + 0.010*"used" + 0.010*"ivy" + 0.009*"property"'),
 (5,
  '0.032*"issue" + 0.021*"user" + 0.015*"version" + 0.012*"commented" + 0.011*"github" + 0.009*"build" + 0.009*"need" + 0.009*"file" +

In [15]:
coherence_model_lda = CoherenceModel(model=lda_model3, texts=final, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.599150531105369
